### Define Constants

In [6]:
from MysqlHelper import MysqlHelper

HOST_NAME = 'localhost'
USERNAME = 'root'
PASSWORD = ''
DB_NAME = 'data_warehouse'
GLOBAL_TABLE_LENGTH = 10

### Create Database and Tables

In [7]:
MysqlHelper.createDatabase(host=HOST_NAME, user=USERNAME, password=PASSWORD, database=DB_NAME)
mysqlHelper = MysqlHelper(host=HOST_NAME, user=USERNAME, password=PASSWORD, database=DB_NAME)

queryString='''
CREATE TABLE time (
    time_key BIGINT(20) NOT NULL AUTO_INCREMENT, 
    time TIME, 
    day INT, 
    day_of_the_week INT, 
    month INT, 
    quarter INT, 
    year INT, 
    PRIMARY KEY (time_key)
);
'''
mysqlHelper.mutate(queryString)

queryString='''
CREATE TABLE item (
    item_key BIGINT(20) NOT NULL AUTO_INCREMENT, 
    item_name VARCHAR(255), 
    brand VARCHAR(255), 
    type VARCHAR(255), 
    supplier_type VARCHAR(255), 
    PRIMARY KEY (item_key)
);
'''
mysqlHelper.mutate(queryString)

queryString='''
CREATE TABLE branch (
    branch_key BIGINT(20) NOT NULL AUTO_INCREMENT, 
    branch_name VARCHAR(255), 
    branch_type VARCHAR(255), 
    PRIMARY KEY (branch_key)
);
'''
mysqlHelper.mutate(queryString)

queryString='''
CREATE TABLE location (
    location_key BIGINT(20) NOT NULL AUTO_INCREMENT, 
    street VARCHAR(255), 
    city VARCHAR(255), 
    state_or_province VARCHAR(255), 
    country VARCHAR(255), 
    PRIMARY KEY (location_key)
);
'''
mysqlHelper.mutate(queryString)

queryString='''
CREATE TABLE sales_fact (
    sales_fact_key BIGINT(20) NOT NULL AUTO_INCREMENT, 
    time_key BIGINT(20), 
    item_key BIGINT(20), 
    branch_key BIGINT(20), 
    location_key BIGINT(20), 
    unit_sold INT, 
    dollars_sold FLOAT, 
    PRIMARY KEY (sales_fact_key),
    FOREIGN KEY (time_key) REFERENCES time(time_key),
    FOREIGN KEY (item_key) REFERENCES item(item_key),
    FOREIGN KEY (branch_key) REFERENCES branch(branch_key),
    FOREIGN KEY (location_key) REFERENCES location(location_key)
);
'''
mysqlHelper.mutate(queryString)

del mysqlHelper

Mutation successful
Mutation successful
Mutation successful
Mutation successful
Mutation successful


### Seeding

Times

In [8]:
from faker import Faker
from random import randint

mysqlHelper = MysqlHelper(host=HOST_NAME, user=USERNAME, password=PASSWORD, database=DB_NAME)
faker = Faker()
dates = [None] * GLOBAL_TABLE_LENGTH

for i in range(GLOBAL_TABLE_LENGTH):
    date = str(faker.date_time_between(start_date="-6y", end_date='now'))
    dates[i] = (date,) * 6

queryString='''
INSERT INTO TIME (
    time,
    day,
    day_of_the_week,
    month,
    quarter,
    year
)
VALUES
(
    TIME(%s), DAY(%s), DAYOFWEEK(%s), MONTH(%s), QUARTER(%s), YEAR(%s)
);
'''

mysqlHelper.mutateMany(queryString, dates)
del mysqlHelper

Mutation successful


Items

In [9]:
from faker import Faker
from random import randint

mysqlHelper = MysqlHelper(host=HOST_NAME, user=USERNAME, password=PASSWORD, database=DB_NAME)
faker = Faker()

valuesList = [None] * GLOBAL_TABLE_LENGTH

for i in range(GLOBAL_TABLE_LENGTH):
    values = {
        'item_name': faker.word(),
        'brand': faker.word(),
        'type': faker.word(),
        'supplier_type': faker.word()
    }
    valuesList[i] = tuple(values.values())

queryString='''
INSERT INTO item (
    item_name,
    brand,
    type,
    supplier_type
)
VALUES
(
    %s, %s, %s, %s
);
'''

mysqlHelper.mutateMany(queryString, valuesList)
del mysqlHelper

Mutation successful


Branch

In [10]:
from faker import Faker
from random import randint

mysqlHelper = MysqlHelper(host=HOST_NAME, user=USERNAME, password=PASSWORD, database=DB_NAME)
faker = Faker()

valuesList = [None] * GLOBAL_TABLE_LENGTH

for i in range(GLOBAL_TABLE_LENGTH):
    values = {
        'branch_name': faker.company(),
        'branch_type': faker.company_suffix(),
    }

    valuesList[i] = tuple(values.values())

queryString='''
INSERT INTO branch (
    branch_name,
    branch_type
)
VALUES
(
    %s, %s
);
'''

mysqlHelper.mutateMany(queryString, valuesList)
del mysqlHelper

Mutation successful


Location

In [11]:
from faker import Faker
from random import randint

mysqlHelper = MysqlHelper(host=HOST_NAME, user=USERNAME, password=PASSWORD, database=DB_NAME)
faker = Faker()

valuesList = [None] * GLOBAL_TABLE_LENGTH

for i in range(GLOBAL_TABLE_LENGTH):
    values = {
        'street': faker.street_name(),
        'city': faker.city(),
        'state_or_province': faker.state(),
        'country': faker.country()
    }
    valuesList[i] = tuple(values.values())

queryString='''
INSERT INTO location (
    street,
    city,
    state_or_province,
    country
)
VALUES
(
    %s, %s, %s, %s
);
'''

mysqlHelper.mutateMany(queryString, valuesList)
del mysqlHelper

Mutation successful


Sales Facts

In [12]:
from faker import Faker
from random import randint

mysqlHelper = MysqlHelper(host=HOST_NAME, user=USERNAME, password=PASSWORD, database=DB_NAME)

rows = mysqlHelper.query(f'SELECT time_key FROM time LIMIT {GLOBAL_TABLE_LENGTH}')
results = mysqlHelper.rowsToList(rows)
timeKeys = [value for sublist in results for value in sublist]

rows = mysqlHelper.query(f'SELECT item_key FROM item LIMIT {GLOBAL_TABLE_LENGTH}')
results = mysqlHelper.rowsToList(rows)
itemKeys = [value for sublist in results for value in sublist]

rows = mysqlHelper.query(f'SELECT branch_key FROM branch LIMIT {GLOBAL_TABLE_LENGTH}')
results = mysqlHelper.rowsToList(rows)
branchKeys = [value for sublist in results for value in sublist]

rows = mysqlHelper.query(f'SELECT location_key FROM location LIMIT {GLOBAL_TABLE_LENGTH}')
results = mysqlHelper.rowsToList(rows)
locationKeys = [value for sublist in results for value in sublist]

valuesList = [None] * GLOBAL_TABLE_LENGTH

for i in range(GLOBAL_TABLE_LENGTH):
    values = {
        'time_key': timeKeys[randint(0, 9)],
        'item_key': itemKeys[randint(0, 9)],
        'branch_key': branchKeys[randint(0, 9)],
        'location_key': locationKeys[randint(0, 9)],
        'unit_sold': randint(0, 50),
        'dollars_sold': randint(50, 1000)
    }
    valuesList[i] = tuple(values.values())

queryString='''
INSERT INTO sales_fact (
    time_key,
    item_key,
    branch_key,
    location_key,
    unit_sold,
    dollars_sold
)
VALUES
(
    %s, %s, %s, %s, %s, %s
);
'''

mysqlHelper.mutateMany(queryString, valuesList)

del mysqlHelper

Mutation successful
